### Borrar y crear widgets

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("catalog", "bakehouse_dev")

catalogName = dbutils.widgets.get("catalog")

### Crear catalogo y bases de datos

In [0]:
try:
    if any(catalog.name == catalogName for catalog in spark.catalog.listCatalogs()):
        spark.sql(f"USE CATALOG {catalogName}")
        print(f"Catalgo actual: {spark.catalog.currentCatalog()}")
    else:
        spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalogName} COMMENT 'Catalogo panaderia'");
except Exception as e:
    msg = f"Error al crear o seleccionar catalogo: {catalogName}. Mensaje: {e}"
    raise Exception(msg)    

In [0]:
try:
    schemaNames = ["raw", "bronze", "silver", "gold"]
    for schemaName in schemaNames:            
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalogName}.{schemaName} COMMENT 'Esquema {schemaName}'")
except Exception as e:
    msg = f"Error al crear esquemas en el catalogo: {catalogName}. Mensaje: {e}"
    raise Exception(msg)

### Crear tablas raw

In [0]:
try:
    schemaName = "raw"
    spark.catalog.setCurrentDatabase(schemaName)
    display(f"Base de datos actual: {spark.catalog.currentDatabase()}")

    tableNames = ["customers", "franchises", "suppliers", "transactions"]
    for tableName in tableNames:
        spark.sql(f"CREATE OR REPLACE TABLE {catalogName}.{schemaName}.{tableName}_{schemaName} AS SELECT * FROM samples.bakehouse.sales_{tableName}")
except Exception as e:
    raise Exception(f"Error al crear tablas enen el catalogo: {catalogName}, esquema: {schemaName}. Mensaje: {e}")

### Crear tablas bronze

In [0]:
try:
    schemaName = "bronze"
    spark.catalog.setCurrentDatabase(schemaName)
    display(f"Base de datos actual: {spark.catalog.currentDatabase()}")
    
    tableNames = ["customers", "franchises", "suppliers", "transactions"]
    for tableName in tableNames:
        spark.sql(f"CREATE OR REPLACE TABLE {catalogName}.{schemaName}.{tableName}_{schemaName} AS SELECT * FROM samples.bakehouse.sales_{tableName}")
except Exception as e:
    raise Exception(f"Error al crear tablas enen el catalogo: {catalogName}, esquema: {schemaName}. Mensaje: {e}")

### Crear tablas silver

In [0]:
try:
    schemaName = "silver"
    spark.catalog.setCurrentDatabase(schemaName)
    display(f"Base de datos actual: {spark.catalog.currentDatabase()}")
    
    tableNames = ["customers", "franchises", "suppliers", "transactions"]
    for tableName in tableNames:
        spark.sql(f"CREATE OR REPLACE TABLE {catalogName}.{schemaName}.{tableName}_{schemaName} AS SELECT * FROM samples.bakehouse.sales_{tableName}")
except Exception as e:
    raise Exception(f"Error al crear tablas enen el catalogo: {catalogName}, esquema: {schemaName}. Mensaje: {e}")

In [0]:
# Obtener el esquema de la tabla origen
df = spark.table("samples.bakehouse.sales_customers")
schema = df.schema

# Construir la definición de columnas con tipos de datos
columns_with_types = ",\n  ".join([f"{field.name} {field.dataType.simpleString()}" for field in schema.fields])
#print (columns_with_types)

# Crear la tabla en el catálogo y esquema destino
spark.sql(f"""
CREATE OR REPLACE TABLE bakehouse_dev.silver.customers_silver (
  customerID bigint,
  first_name string,
  last_name string,
  gender boolean,
  email_address string,
  phone_number string,
  address string,
  continent string,
  country string,
  state string,
  city string,
  postal_zip_code bigint,
  updatedate timestamp
)
""")

## Crear tablas gold

In [0]:
try:
    schemaName = "gold"
    spark.catalog.setCurrentDatabase(schemaName)
    display(f"Base de datos actual: {spark.catalog.currentDatabase()}")
    
    tableNames = ["franchises", "suppliers", "transactions"]
    for tableName in tableNames:
        spark.sql(f"CREATE OR REPLACE TABLE {catalogName}.{schemaName}.{tableName}_{schemaName} AS SELECT * FROM samples.bakehouse.sales_{tableName}")
except Exception as e:
    raise Exception(f"Error al crear tablas enen el catalogo: {catalogName}, esquema: {schemaName}. Mensaje: {e}")

In [0]:
# Obtener el esquema de la tabla origen
df = spark.table("samples.bakehouse.sales_customers")
schema = df.schema

# Construir la definición de columnas con tipos de datos
columns_with_types = ",\n  ".join([f"{field.name} {field.dataType.simpleString()}" for field in schema.fields])
#print (columns_with_types)

# Crear la tabla en el catálogo y esquema destino
spark.sql(f"""
CREATE OR REPLACE TABLE bakehouse_dev.gold.customers_gold (
  country string,
  customer_count_country long,
  state string,
  customer_count_country_state long,
  updatedate timestamp
)
""")